# **Group 5 ED(C) Deep Learning Approach with the use of Transformers - Demo on the test set**

Import necessary libraries

In [ ]:
from claims_dataset import ClaimEvidenceDataset
from evidence_classifier import EvidenceClassifier
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader

Load the dev set

In [ ]:
test_df = pd.read_csv('data/test.csv')
test_df.head()

Load the model and output its structure

The 'ED_model_C.pth' model can be downloaded from the following link: https://drive.google.com/drive/folders/1TWv5UKNsNeQGxafx3GQf87Dc8vcp5V8t?usp=sharing

In [ ]:
model_path = 'ED_model_C.pth'
model = EvidenceClassifier()
model.load_state_dict(torch.load(model_path))
print(model)

Define the predict function

In [ ]:
def predict(test_csv, model_path):
  # Set device to cuda to run on GPU
  device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
  # Load the model state dictionary
  model = EvidenceClassifier().to(device)
  model.load_state_dict(torch.load(model_path, map_location=device))

  # load the data to evaluate on (in this case the dev dataset)
  test_dataset = ClaimEvidenceDataset(test_csv)
  test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
  # set the model to evaluation mode
  model.eval()

  all_predictions = []
  all_probabilities = []

  with torch.no_grad():
    for batch in test_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      token_type_ids = batch['token_type_ids'].to(device)
      labels = batch['labels'].to(device)

      logits = model(input_ids, attention_mask, token_type_ids)
      probabilities = torch.nn.functional.softmax(logits, dim=1)
      predictions = torch.argmax(probabilities, dim=1)

      all_predictions.extend(predictions.cpu().numpy())
      all_probabilities.extend(probabilities.cpu().numpy())


  df = pd.read_csv(test_csv)
  df['prediction'] = all_predictions
  return df

Run the model in inference mode on the test set and print first few predictions

In [ ]:
predictions = predict('data/test.csv', 'best_model.pth')

for i in range(10):
  print(f"Evidence: {predictions['evidence'][i]}")
  print(f"Claim: {predictions['claim'][i]}")
  print(f"Label: {predictions['label'][i]}")
  print(f"Prediction: {predictions['prediction'][i]}")

Save the predictions

In [ ]:
predictions['prediction'].to_csv('Group_5_C.csv', index=False)

Run the model on User Inputs

In [ ]:
claim = input("Enter a claim: ")
evidence = input("Enter an evidence: ")

df = pd.DataFrame({'claim': [claim], 'evidence': [evidence]})
df.to_csv('data/input.csv', index=False)
predictions = predict('data/input.csv', 'best_model.pth')
prediction = "relevant" if predictions['prediction'][0] == 1 else "irrelevant"
print(f"Claim: {predictions['claim'][0]}")
print(f"Evidence: {predictions['evidence'][0]}")
print(f"Prediction: {prediction}")